<a href="https://colab.research.google.com/github/DuyguA/data_science_scripts/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
import pandas as pd

TensorFlow 2.x selected.
2.1.0


In [ ]:
!sudo pip install transformers

     |████████████████████████████████| 481kB 2.8MB/s 
     |████████████████████████████████| 870kB 13.8MB/s 
     |████████████████████████████████| 1.0MB 11.7MB/s 
     |████████████████████████████████| 3.1MB 22.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=608c78fbf38cd7961259a0f7b60918dfe1c575b023f060c8d5aaaa4edc43fbdf
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
btokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
bmodel = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving sentiment_labeled_data.xlsx to sentiment_labeled_data.xlsx


In [ ]:
import io
data = pd.read_excel(io.BytesIO(uploaded['sentiment_labeled_data.xlsx']), converters={"OTHER":int})


In [ ]:
sentences = []
y = []

In [ ]:
for index, row in data.iterrows():
    sentences.append(row["ORJINAL"])
    y.append(row["SORU"])

In [ ]:
w =[btokenizer.encode(x, add_special_tokens=True, max_length=120, pad_to_max_length=True) for x in sentences]


In [ ]:
input_layer = tf.keras.layers.Input(shape = (120,), dtype='int32')
bert = bmodel(input_layer)
bert = bert[0]

#conv=tf.keras.layers.Conv1D(128, kernel_size=5, activation='relu', name="input")(bert)
#pooling = tf.keras.layers.GlobalMaxPooling1D()(bert)
#lstm = tf.keras.layers.LSTM(units=64, dropout=0.3, return_sequences=False)(bert)
dense = tf.keras.layers.Dense(64, activation='relu')(bert)
output = tf.keras.layers.Dense(1, activation="relu")(dense)
classifier= tf.keras.layers.Dense(1, activation='relu', name='outputs')(output)

#dropout = tf.keras.layers.Dropout(0.1)(bert)
#flat = tf.keras.layers.Flatten()(dropout)
#classifier = tf.keras.layers.Dense(units=1)(flat)

keras_model = tf.keras.Model(inputs=input_layer, outputs=classifier)
keras_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 120)]             0         
_________________________________________________________________
tf_bert_for_sequence_classif ((None, 2),)              177854978 
_________________________________________________________________
dense_2 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
_________________________________________________________________
outputs (Dense)              (None, 1)                 2         
Total params: 177,855,237
Trainable params: 177,855,237
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
keras_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
keras_model.fit(w, y, epochs=10, batch_size=32)

Train on 4957 samples
Epoch 1/10
4957/4957 [==============================] - 6192s 1s/sample - loss: 6.2795 - accuracy: 0.5929
Epoch 2/10
4957/4957 [==============================] - 6167s 1s/sample - loss: 6.2795 - accuracy: 0.5929
Epoch 3/10
1760/4957 [=========>....................] - ETA: 1:06:14 - loss: 6.1525 - accuracy: 0.6011